In [ ]:
import sys
import os
import gtools
from tqdm import tqdm
import numpy as np
import argparse

In [ ]:
parser = argparse.ArgumentParser(description='''
Adds columns to the duplex-seq variants generated by duplex_caller.py which are required for filtering with filter_duplex_variants.py.|n

Requires the following as input:|n
    - duplex variant tsv: generated by running duplex_caller.py on the sample to call variants within|n
    - variant frequency tsv: generated by running dup_informed_caller.py --duplicate_support 2 on the same sample sample as the duplex variant tsv. This will be 
        used to calculate the fraction of molecules in the sample which support each variant|n
    - control tsvs: generated by running dup_informed_caller.py --duplicate_support 2 --min_support 2 on different samples. This will be used to filter out any 
        variants which occur too frequently in these samples|n
    - blacklists: generated by generate_duplex_blacklists.py|n

Will add the following columns to the duplex variant tsv:|n
    - high_sup: number of supporting bases with BQ>=30|n
    - avg_mq: average MQ of supporting reads|n
    - cov_per: coverage percentile of the variant site, uses blacklist (e.g. 0.02 means 2% of the genome has a lower read coverage than the variant position)|n
    - dup_dist: minimum hamming distance between the fragment containing the variant and another genomic position, uses blacklist (e.g. 2 means there's another kmer 
    in the genome that's 2 edits away from the kmer starting at the fragment start (the value of "k" is defined when blacklists are generated and is not necessarily
    the fragment length))|n
    - poly_at: length of longest poly-A/T repeat ending at or 1bp away from the variant position, uses blacklist (e.g. 5 means the variant is at the end of a 5bp 
    poly-A or poly-T repeat)|n
    - poly_rep: length of longest poly-di/trinucleotide repeat ending at or 1bp away from the variant position, uses blacklist (e.g. 4 means the variant is at the  
    end of a length 3 dinucleotide or trinucleotide repeat (e.g. CCTCCTCCT))|n
    - sample_sup: number of fragments in the sample which support the variant, uses the variant frequency tsv|n
    - sample_cov: number of fragments in the sample which overlap the variant position, uses the variant frequency tsv|n
    - control_sup: number of fragments in the control samples which support the variant, uses the control tsvs for this|n
    ''', formatter_class=gtools.EscapeFormatter)
parser.add_argument('-i', '--input', required=True, dest='duplex_tsv', metavar='TSV', type=str,
                   help='generated by running duplex_caller.py on the sample to call variants within. Must be sorted by chrom-pos-ref-alt, \
                   which duplex_caller.py does by default.')
parser.add_argument('-f', '--frequency', required=True, dest='frequency_tsv', metavar='TSV', type=str,
                   help='generated by running dup_informed_caller.py --duplicate_support 2 on the same sample sample as --input. Must be \
                   sorted by chrom-pos-ref-alt, which dup_informed_caller.py does by default.')
parser.add_argument('-c', '--controls', required=True, dest='control_tsvs', metavar='TSV', type=str, nargs='+',
                   help='generated by running dup_informed_caller.py --duplicate_support 2 --min_support 2 on samples other than --input')
parser.add_argument('-b', '--blacklists', required=True, dest='blacklist_prefix', metavar='PREFIX', type=str, 
                   help='prefix of the blacklist files, same as was passed to generate_duplex_blacklists --prefix. Will look for {prefix}{chrom}_{blacklist type}.npy, \
                   where blacklist type is coverage_percentile, duplicate_distance, poly_at_length, and poly_repeat_length.')
parser.add_argument('-o', '--output', dest='output_tsv', metavar='TSV', type=str,
                   help='output tsv. Will have the same variants as --input but with extra columns added (default: stdout)')

if gtools.running_notebook(): # run this if in a jupyter notebook
    print('Determined code is running in Jupyter')
    if os.getcwd()[:8] != '/scratch': # switch to the scratch directory where all the data files are
        os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')
    args = parser.parse_args('-i data/variant/big_Col-0-1_duplex_strandsup.tsv -o tmp/test_add_duplex.tsv -f data/variant/big_Col-0-1_informed_freq.tsv -b data/blacklist/arabidopsis/duplex_blacklist_ -c data/variant/big_Col-0-2_informed_control.tsv data/variant/big_Col-0-3_informed_control.tsv'.split()) # used for testing
else: # run this if in a terminal
    args = parser.parse_args()

if args.output_tsv is not None and '/' in args.output_tsv:
    os.makedirs(os.path.dirname(args.output_tsv), exist_ok=True)
sys.stderr.write('Running add_duplex_filter_columns.py with arguments:\n' + '\n'.join([f'{key}={val}' for key, val in vars(args).items()]) + '\n')

# Load blacklists

In [ ]:
# find all chromosomes in the input
sys.stderr.write('finding all chromosomes in input tsv\n')
chroms = set()
total_lines = 0
with open(args.duplex_tsv, 'r') as f:
    next(f) # skip header
    for l in f:
        chroms.add(l.split('\t', 1)[0])
        total_lines += 1

In [ ]:
sys.stderr.write('loading blacklists\n')

# all are indexed as bl[chrom][pos] = value
bl_cov_per = dict() # coverage percentile
bl_dup_dist = dict() # mismatches required to have a duplicate 120mer in the genome
bl_poly_at = dict() # length of poly-A or poly-T starting or ending at position
bl_poly_rep = dict() # length of di or trinucleotide repeat starting or ending at position
for chrom in tqdm(chroms):
    bl_cov_per[chrom] = np.load(f'{args.blacklist_prefix}{chrom}_coverage_percentile.npy')
    bl_dup_dist[chrom] = np.load(f'{args.blacklist_prefix}{chrom}_duplicate_distance.npy')
    bl_poly_at[chrom] = np.load(f'{args.blacklist_prefix}{chrom}_poly_at_length.npy')
    bl_poly_rep[chrom] = np.load(f'{args.blacklist_prefix}{chrom}_poly_repeat_length.npy')    

In [ ]:
def line_to_var(line):
    var = line[:-1].split('\t')
    var[1] = int(var[1]) # change position to an int
    return var

In [ ]:
# def add_mut_info(call_file, freq_file, control_files, output_file):

# open all files
f_call = open(args.duplex_tsv, 'r')
f_freq = open(args.frequency_tsv, 'r')
f_conts = [open(f, 'r') for f in args.control_tsvs]
f_out = open(args.output_tsv, 'w') if args.output_tsv is not None else sys.stdout

# extract the header from all files
try:
    dcs = {x:i for i, x in enumerate(next(f_call)[:-1].split('\t'))} # converts a column name to column index for duplex_caller tsv
    ics = {x:i for i, x in enumerate(next(f_freq)[:-1].split('\t'))} # converts a column name to column index for dup_informed_caller tsv
    [next(f) for f in f_conts]
    
    # write the output header with extra columns
    f_out.write('\t'.join(list(dcs.keys()) + 'high_sup avg_mq cov_per dup_dist poly_at poly_rep sample_sup sample_cov control_sup'.split()) + '\n')

    # load the first freq and control variants
    cur_freq = line_to_var(next(f_freq))
    cur_conts = [line_to_var(next(f)) for f in f_conts]
    
except StopIteration as e:
    sys.stderr.write(f'ERROR: one of the input tsvs is empty\n')
    sys.stderr.write(e)
    exit()

# for each variant in the call file, add extra information and output
bar = tqdm(total=total_lines, miniters=10000, maxinterval=999)
while True:
    # get the next variant in the call file
    try:
        cur_var = line_to_var(next(f_call))
    except StopIteration:
        break

    # calculate the number of supporting reads with a high BQ value (>=30)
    if cur_var[dcs['ref']] == '*' or cur_var[dcs['alt']] == '*':
        high_sup = int(cur_var[dcs['f_sup']]) + int(cur_var[dcs['r_sup']])
    else:
        high_sup = sum([(ord(bq) - 33) >= 30 for bq in cur_var[dcs['bq']]])

    # calculate the average MQ of supporting reads
    avg_mq = sum([ord(mq) - 33 for mq in cur_var[dcs['mq']]]) / len(cur_var[dcs['mq']])

    # find the blacklist values
    cov_per = bl_cov_per[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
    dup_dist = bl_dup_dist[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
    poly_at = bl_poly_at[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
    poly_rep = bl_poly_rep[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]

    # read in variants from the frequency and control files until at or past the current variant
    while cur_freq[:4] < cur_var[:4]:
        try:
            cur_freq = line_to_var(next(f_freq))
        except StopIteration:
            cur_freq = [chr(1000)]

    for i in range(len(f_conts)):
        while cur_conts[i][:4] < cur_var[:4]:
            try:
                cur_conts[i] = line_to_var(next(f_conts[i]))
            except StopIteration:
                cur_conts[i] = [chr(1000)]

    # add the support and coverage of the variant within the sample
    if cur_freq[:4] == cur_var[:4]:
        sample_sup = int(cur_freq[ics['f_sup']]) + int(cur_freq[ics['r_sup']])
        sample_cov = int(cur_freq[ics['f_cov']]) + int(cur_freq[ics['r_cov']])
    else:
        sample_sup = 0
        sample_cov = 0

    # add the support of the variant across all control samples
    control_sup = 0
    for i in range(len(f_conts)):
        if cur_conts[i][:4] == cur_var[:4]:
            control_sup += int(cur_conts[i][ics['f_sup']]) + int(cur_conts[i][ics['r_sup']])

    cur_var += [high_sup, avg_mq, cov_per, dup_dist, poly_at, poly_rep, sample_sup, sample_cov, control_sup]

    if True:
        f_out.write('\t'.join(map(str, cur_var)) + '\n')

    bar.update()

f_call.close()
f_freq.close()
for f in f_conts:
    f.close()
f_out.close()

In [ ]:
sys.stderr.write(f'completed add_duplex_filter_columns.py on {args.duplex_tsv}\n')